In [1]:
import sys
import logging
import numpy as np
from joblib import hash, dump
import os

from deer.default_parser import process_args
from deer.agent import NeuralAgent
from deer.learning_algos.CRAR_torch import CRAR
from catcher_env import MyEnv as catcher_env
import deer.experiment.base_controllers as bc

from deer.policies import EpsilonGreedyPolicy

In [2]:
class Defaults:
    # ----------------------
    # Experiment Parameters
    # ----------------------
    steps_per_epoch = 2000
    epochs = 50
    steps_per_test = 500
    period_btw_summary_perfs = 1
    
    # ----------------------
    # Environment Parameters
    # ----------------------
    frame_skip = 2

    # ----------------------
    # DQN Agent parameters:
    # ----------------------
    update_rule = 'rmsprop'
    learning_rate = 0.0005
    learning_rate_decay = 0.9
    discount = 0.9
    discount_inc = 1
    discount_max = 0.99
    rms_decay = 0.9
    rms_epsilon = 0.0001
    momentum = 0
    clip_norm = 1.0
    epsilon_start = 1.0
    epsilon_min = 1.0
    epsilon_decay = 10000
    update_frequency = 1
    replay_memory_size = 1000000
    batch_size = 32
    freeze_interval = 1000
    deterministic = False

HIGHER_DIM_OBS = True
HIGH_INT_DIM = False

In [3]:
parameters = Defaults()
if parameters.deterministic:
    rng = np.random.RandomState(123456)
else:
    rng = np.random.RandomState()

In [4]:
# --- Instantiate environment ---
env = catcher_env(rng, higher_dim_obs=HIGHER_DIM_OBS, reverse=False)

# --- Instantiate learning algorithm ---
learning_algo = CRAR(
    env,
    parameters.rms_decay,
    parameters.rms_epsilon,
    parameters.momentum,
    parameters.clip_norm,
    parameters.freeze_interval,
    parameters.batch_size,
    parameters.update_rule,
    rng,
    double_Q=True,
    high_int_dim=HIGH_INT_DIM,
    internal_dim=3)

test_policy = EpsilonGreedyPolicy(learning_algo, env.nActions(), rng, 0.1)#1.)

# --- Instantiate agent ---
agent = NeuralAgent(
    env,
    learning_algo,
    parameters.replay_memory_size,
    max(env.inputDimensions()[i][0] for i in range(len(env.inputDimensions()))),
    parameters.batch_size,
    rng,
    test_policy=test_policy)

# --- Create unique filename for FindBestController ---
h = hash(vars(parameters), hash_name="sha1")
fname = "test_" + h
print("The parameters hash is: {}".format(h))
print("The parameters are: {}".format(parameters))

# --- Bind controllers to the agent ---
# Before every training epoch (periodicity=1), we want to print a summary of the agent's epsilon, discount and 
# learning rate as well as the training epoch number.
agent.attach(bc.VerboseController(
    evaluate_on='epoch', 
    periodicity=1))

# As for the discount factor and the learning rate, one can update periodically the parameter of the epsilon-greedy
# policy implemented by the agent. This controllers has a bit more capabilities, as it allows one to choose more
# precisely when to update epsilon: after every X action, episode or epoch. This parameter can also be reset every
# episode or epoch (or never, hence the resetEvery='none').
agent.attach(bc.EpsilonController(
    initial_e=parameters.epsilon_start, 
    e_decays=parameters.epsilon_decay, 
    e_min=parameters.epsilon_min,
    evaluate_on='action',
    periodicity=1,
    reset_every='none'))

# During training epochs, we want to train the agent after every [parameters.update_frequency] action it takes.
# Plus, we also want to display after each training episode (!= than after every training) the average bellman
# residual and the average of the V values obtained during the last episode, hence the two last arguments.
agent.attach(bc.TrainerController(
    evaluate_on='action', 
    periodicity=parameters.update_frequency, 
    show_episode_avg_V_value=True, 
    show_avg_Bellman_residual=True))

# Every epoch end, one has the possibility to modify the learning rate using a LearningRateController. Here we 
# wish to update the learning rate after every training epoch (periodicity=1), according to the parameters given.
agent.attach(bc.LearningRateController(
    initial_learning_rate=parameters.learning_rate, 
    learning_rate_decay=parameters.learning_rate_decay,
    periodicity=1))

# Same for the discount factor.
agent.attach(bc.DiscountFactorController(
    initial_discount_factor=parameters.discount, 
    discount_factor_growth=parameters.discount_inc, 
    discount_factor_max=parameters.discount_max,
    periodicity=1))

# All previous controllers control the agent during the epochs it goes through. However, we want to interleave a 
# "validation epoch" between each training epoch ("one of two epochs", hence the periodicity=2). We do not want 
# these validation epoch to interfere with the training of the agent, which is well established by the 
# TrainerController, EpsilonController and alike. Therefore, we will disable these controllers for the whole 
# duration of the validation epochs interleaved this way, using the controllersToDisable argument of the 
# InterleavedTestEpochController. For each validation epoch, we want also to display the sum of all rewards 
# obtained, hence the showScore=True. Finally, we want to call the summarizePerformance method of ALE_env every 
# [parameters.period_btw_summary_perfs] *validation* epochs.
agent.attach(bc.InterleavedTestEpochController(
    id=catcher_env.VALIDATION_MODE, 
    epoch_length=parameters.steps_per_test,
    periodicity=1,
    show_score=True,
    summarize_every=1))

The parameters hash is: 62977be8e45d8a56a5537c11dfd5d2fd8dda69e0
The parameters are: <__main__.Defaults object at 0x2ab37d83c070>


In [ ]:
# --- Run the experiment ---
try:
    os.mkdir("params")
except Exception:
    pass
dump(vars(parameters), "params/" + fname + ".jldump")
agent.run(parameters.epochs, parameters.steps_per_epoch)

Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([ 0.0865, -0.0777,  0.0089], device='cuda:0') tensor([ 0.0004, -0.3450,  0.1292], device='cuda:0') tensor([ 0.0884, -0.0781,  0.0093], device='cuda:0')
R[0]
tensor([0.1490], device='cuda:0', grad_fn=<SelectBackward0>)
Average (on the epoch) training loss: 0.000763518393796403
Episode average V value: 0
Average (on the epoch) training loss: 0.02487533270066385
Episode average V value: 0
Average (on the epoch) training loss: 0.05749703552618694
Episode average V value: 0
Average (on the epoch) training loss: 0.06941996287579448
Episode average V value: 0
Average (on the epoch) training loss: 0.07434924849836864
Episode average V value: 0
Average (on the epoch) training loss: 0.07865321528323584
Episode average V value: 0
Average (on the epoch) training loss: 0.07765003541017874
Episode average V value: 0
Average (on the epoch) training loss: 0.079010212

Average (on the epoch) training loss: 0.08794319180843091
Episode average V value: 0
Average (on the epoch) training loss: 0.08782558719282507
Episode average V value: 0
Average (on the epoch) training loss: 0.08751680248458797
Episode average V value: 0
Average (on the epoch) training loss: 0.0872155250870467
Episode average V value: 0
Average (on the epoch) training loss: 0.08721183990180639
Episode average V value: 0
Average (on the epoch) training loss: 0.08719424352635353
Episode average V value: 0
Average (on the epoch) training loss: 0.08704977412970943
Episode average V value: 0
Average (on the epoch) training loss: 0.08693071422289393
Episode average V value: 0
Average (on the epoch) training loss: 0.08676201324246809
Episode average V value: 0
Average (on the epoch) training loss: 0.08652666857049475
Episode average V value: 0
Average (on the epoch) training loss: 0.08668105523332613
Episode average V value: 0
Average (on the epoch) training loss: 0.08672598168549826
Episode 

Average (on the epoch) training loss: 0.08160727437877925
Episode average V value: 0
Average (on the epoch) training loss: 0.08181859940367922
Episode average V value: 0
Average (on the epoch) training loss: 0.08169984021537696
Episode average V value: 0
Average (on the epoch) training loss: 0.08150728582725332
Episode average V value: 0
Average (on the epoch) training loss: 0.08156318493661086
Episode average V value: 0
Average (on the epoch) training loss: 0.08122328250883099
Episode average V value: 0
Average (on the epoch) training loss: 0.0811222694964578
Episode average V value: 0
Average (on the epoch) training loss: 0.0810990898489443
Episode average V value: 0
Average (on the epoch) training loss: 0.08103480638036797
Episode average V value: 0
Average (on the epoch) training loss: 0.0809798313555599
Episode average V value: 0
Average (on the epoch) training loss: 0.08094927277684019
Episode average V value: 0
Average (on the epoch) training loss: 0.08078672311487241
Episode av

/home/cf2794/.conda/envs/auxrl/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/share/ctn/users/cf2794/Code/deer/deer/learning_algos/CRAR_torch.py:379: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  state = torch.as_tensor(state, device=self.device).float()


Testing score per episode (id: 0) is -0.48214285714285715 (average over 56 episode(s))
== Mean score per episode is -0.4821419961750068 over 56 episodes ==


...

Average (on the epoch) training loss: 0.007339057064012787
Episode average V value: 0
Average (on the epoch) training loss: 0.007338711155797701
Episode average V value: 0
Average (on the epoch) training loss: 0.007330146098959874
Episode average V value: 0
Average (on the epoch) training loss: 0.007332021654647055
Episode average V value: 0
Average (on the epoch) training loss: 0.007327996573690084
Episode average V value: 0
Average (on the epoch) training loss: 0.007328774944526803
Episode average V value: 0
Average (on the epoch) training loss: 0.007323895799921466
Episode average V value: 0
Average (on the epoch) training loss: 0.007322860834085279
Episode average V value: 0
Average (on the epoch) training loss: 0.007322393646146255
Episode average V value: 0
Average (on the epoch) training loss: 0.007323909456194457
Episode average V value: 0
Average (on the epoch) training loss: 0.007325779184752542
Episode average V value: 0
Average (on the epoch) training loss: 0.00732696397099

/share/ctn/users/cf2794/Code/deer/examples/test_CRAR/catcher_env.py:184: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  line3 = ax.scatter(


self.loss_T, self.loss_R, self.loss_gamma, self.loss_Q, self.loss_disentangle_t, self.loss_disambiguate1, self.loss_disambiguate2
0.00735939879482612 0.010951095759868622 0.0002716373152370579 0.00676704186713323 0.07121684244275094 0.00016200490295886994 0.026232944869436323
self.loss_interpret/500.
0.0
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 False
Es[0], TEs[0], Esp_[0]
tensor([0.9942, 0.5449, 0.4699], device='cuda:0') tensor([ 0.9872, -0.1004,  0.8990], device='cuda:0') tensor([ 0.9395, -0.1557,  0.9520], device='cuda:0')
R[0]
tensor([0.0110], device='cuda:0', grad_fn=<SelectBackward0>)
Average (on the epoch) training loss: 0.006719429563317034
Episode average V value: 0
Average (on the epoch) training loss: 0.007603107304829691
Episode average V value: 0
Average (on the epoch) training loss: 0.007824183182997836
Episode average V value: 0
Average (on the epoch) training loss: 0.007502015848230157
Episode average V value: 

Average (on the epoch) training loss: 0.006888516711336861
Episode average V value: 0
Average (on the epoch) training loss: 0.006887583093917083
Episode average V value: 0
Average (on the epoch) training loss: 0.00689798329739895
Episode average V value: 0
Average (on the epoch) training loss: 0.006885326816084445
Episode average V value: 0
Average (on the epoch) training loss: 0.006871936892408853
Episode average V value: 0
Average (on the epoch) training loss: 0.006860777366610403
Episode average V value: 0
Average (on the epoch) training loss: 0.006862795694955978
Episode average V value: 0
Average (on the epoch) training loss: 0.006851610809040134
Episode average V value: 0
Average (on the epoch) training loss: 0.006841284802216065
Episode average V value: 0
Average (on the epoch) training loss: 0.006847563796435242
Episode average V value: 0
Average (on the epoch) training loss: 0.006851225323242764
Episode average V value: 0
Average (on the epoch) training loss: 0.006862469538843

Average (on the epoch) training loss: 0.00694702539476566
Episode average V value: 0
Average (on the epoch) training loss: 0.006944371112741721
Episode average V value: 0
Average (on the epoch) training loss: 0.00694883721621914
Episode average V value: 0
Average (on the epoch) training loss: 0.006947967777382086
Episode average V value: 0
Average (on the epoch) training loss: 0.006938088055367964
Episode average V value: 0
Average (on the epoch) training loss: 0.006940543449106378
Episode average V value: 0
Average (on the epoch) training loss: 0.006944574377027318
Episode average V value: 0
Average (on the epoch) training loss: 0.00694051811102009
Episode average V value: 0
Average (on the epoch) training loss: 0.0069380195883709484
Episode average V value: 0
Average (on the epoch) training loss: 0.006933277502962331
Episode average V value: 0
